In [17]:
import sys
import json

params = {'general':{'periods':['test']}}
default = {'scenario':'orchestrator','training_folder': '../..', 'params': params} # Default execution parameters
# here sceneario: demand is only use not on lambda.
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'scenario': 'orchestrator', 'training_folder': '../..', 'params': {'general': {'periods': ['test']}}}


In [18]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel


In [19]:
periods = argv['params']['general']['periods']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'
if on_lambda:
    bucket_name = 'quetzal-immense' #os.environ.get('BUCKET_NAME')
    output_folder = training_folder + '/outputs/'
    scenario_folders = []
    input_folders = []
    for p in periods:
        folder = 's3://' + bucket_name + '/' + p + '/outputs/'
        scenario_folders.append(folder)
        input_folders.append(input_folder + p + '/')

else:
    output_folder = training_folder + '/scenarios/' + argv['scenario'] + '/outputs/'
    scenario_folders = []
    input_folders = []
    for p in periods:
        folder = training_folder + '/scenarios/' + p + '/outputs/'
        scenario_folders.append(folder)
        input_folders.append(input_folder + p + '/')
    
print('input folder: ', input_folders)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folders)

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ['../../inputs/test/']
output folder:  ../../scenarios/orchestrator/outputs/
scen folder :  ['../../scenarios/test/outputs/']


In [20]:
#var, ancestry = excel.read_var(file=input_folder+'/parameters.xlsx', scenario=period, return_ancestry=True)


# modal shift

In [83]:
#TODO 12.3.2 Incremental or Pivot-point Modelling

In [85]:
def get_pt_part(pt_los:pd.DataFrame, 
                car_los:pd.DataFrame, 
                method:str='kirchhoff',
                power:float=-1.42,
                part_max:float=1) -> pd.DataFrame:
    
    cols = ['origin','destination','time']
    los = pt_los[cols].merge(car_los[cols],on=['origin','destination'],how='inner')
    los = los.rename(columns = {'time_x':'time_pt','time_y':'time_car'})
    los['time_ratio'] = los['time_pt'] / los['time_car']
    if method == 'kirchhoff':
        los['pt_part'] = los['time_ratio'].apply(lambda x: min(x**(power), part_max))
    else:
        print('method not recognized')
    return los


In [93]:
for period, ref_folder, scen_folder in zip(periods,input_folders,scenario_folders):
    print(ref_folder)
    ref_pt_skim = pd.read_csv(ref_folder+'skims/pt_skim.csv')
    ref_car_skim = pd.read_csv(ref_folder+'skims/car_skim.csv')
    pt_skim = pd.read_csv(scen_folder+'pt_skim.csv')
    car_skim = pd.read_csv(scen_folder+'car_skim.csv')
    volumes = pd.read_csv(ref_folder+'volumes/volumes.csv')
    volumes = volumes.set_index('index')

    ref_pt_part = get_pt_part(ref_pt_skim, ref_car_skim)
    pt_part = get_pt_part(pt_skim, car_skim)
    
    ref_pt_part = ref_pt_part.set_index(['origin','destination'])['pt_part'].to_dict()
    pt_part['ref_pt_part'] = pt_part.set_index(['origin','destination']).index.map(ref_pt_part.get)
    pt_part['transfert_prob'] = pt_part['pt_part'] - pt_part['ref_pt_part']
    
    transfert_prob = pt_part.set_index(['origin','destination'])['transfert_prob'].to_dict()
    volumes['transfert_prob'] = volumes.set_index(['origin','destination']).index.map(transfert_prob.get)
    volumes['transfert'] = volumes['pt'] * volumes['transfert_prob']
    volumes['pt'] = volumes['pt'] + volumes['transfert']
    volumes['car'] = volumes['pt'] - volumes['transfert']
    
    volumes[['origin','destination','pt','car']].to_csv(output_folder+'volume_'+period+'.csv')

../../inputs/test/
